In [49]:
import pandas as pd
import strym
from strym import strymread
from strym import strymmap
import matplotlib.pyplot as plt
from pylab import rcParams
import strym.DBC_Read_Tools as dbc
import numpy as np

In [50]:
CAN_path = '../data/publishable-circles/2T3MWRFVXLW056972/libpanda/2021_03_09/2021-03-09-00-50-07_2T3MWRFVXLW056972_CAN_Messages.csv'
r = strymread(csvfile=CAN_path)

GPS_path = '../data/publishable-circles/2T3MWRFVXLW056972/libpanda/2021_03_09/2021-03-09-00-50-07_2T3MWRFVXLW056972_GPS_Messages.csv'
strymGPS = strymmap(csvfile=GPS_path)
GPSRecords = strymGPS.dataframe
GPSRecords = GPSRecords.sort_values(by="Systime")

# Test to be sure of overlap
earliestGPS = GPSRecords['Systime'][0]
latestGPS = GPSRecords['Systime'][-1]
earliestCAN = r.dataframe['Time'][0]
latestCAN = r.dataframe['Time'][-1]

if not ((earliestGPS >= earliestCAN and earliestGPS <= latestCAN) or (latestGPS >= earliestCAN and latestGPS <= latestCAN)):
    print("These files cannot be linked as there is no overlap")

[2021_07_12_09_19_09] (root) INFO: Vehicle model infered is toyota-rav4-2020
[2021_07_12_09_19_43] (cantools.database.can.database) WARNING: Overwriting message 'GAS_PEDAL' with 'GAS_PEDAL' in the name to message dictionary.
[2021_07_12_09_19_43] (root) INFO: Reading GPS file ../data/publishable-circles/2T3MWRFVXLW056972/libpanda/2021_03_09/2021-03-09-00-50-07_2T3MWRFVXLW056972_GPS_Messages.csv
GPS signal first acquired at 2021-03-08 19:50:19:300000


In [51]:
time = r.dataframe['Time']#.setIndex('Time')
bus = r.dataframe['Bus']#.setIndex('Time')
messageID = r.dataframe['MessageID']#.setIndex('Time')
message = r.dataframe['Message']#.setIndex('Time')
messageLength = r.dataframe['MessageLength']#.setIndex('Time')

In [52]:
def interpolate(time, preTime, postTime, preValue, postValue):
    d1 = abs(time - preTime)
    w1 = 1/d1
    d2 = abs(time - postTime)
    w2 = 1/d2
    wv1 = preValue * w1
    wv2 = postValue * w2
    weightedValueSum = wv1 + wv2
    weightSum = w1 + w2
    average = weightedValueSum / weightSum
    return average

In [53]:
def mostRelevantRecord(records, referenceTime):
    # records is the series stats[key]
    # referenceTime is the current GPS Systime
    mostRelevant = None
    try:
        mostRelevant = records.iloc[records.index.get_loc(referenceTime)]
    except:
    #     print("interpolating")
        earliestTime = records[0]#.iloc[0].name
        latestTime = records[-1]#.iloc[-1].name
        if referenceTime < earliestTime or referenceTime > latestTime:
            if referenceTime < earliestTime:
                print(referenceTime, "<", earliestTime)
            if referenceTime > latestTime:
                print(referenceTime, ">", earliestTime)
        else:
            # finds previous value if no exact match
            pre_records = records.loc[~records.index.duplicated(keep='last')]
            preTime = pre_records[pd.Index(pre_records).get_loc(referenceTime, method="pad")]
            
            # finds next closest index if no exact match
            post_records = records.loc[~records.index.duplicated(keep='first')]
            postTime = post_records[pd.Index(post_records).get_loc(referenceTime, method="backfill")]

            #mostRelevant = interpolate(interpolate(time, preTime, postTime, preValue, postValue))
            
    average = (postTime + preTime) / 2
    
    return average # referenceTime, pre.name, post.name, pre.Message, post.Message

In [ ]:
columns = {"Time":[], "Longitude":[], "Latitude":[]}
stats = {"Time":time, "Bus":bus, "MessageID":messageID, "Message":message, "MessageLength":messageLength}
for key in stats:
    print("Now linking", key)
    columns[key] = []
    counter = 0
    for GPSRecord in GPSRecords.iterrows():
        counter+=1
        if counter % 10 == 0:
            print("Linked", counter, "records")
        interpolatedRecord = mostRelevantRecord(stats[key], GPSRecord[1]['Systime'])
        columns[key].append(interpolatedRecord)
        if len(columns[key]) > len(columns["Longitude"]):
            columns["Time"].append(GPSRecord[1].Systime)
            columns["Longitude"].append(GPSRecord[1]["Long"])
            columns["Latitude"].append(GPSRecord[1]["Lat"])

Now linking Time
Linked 10 records
Linked 20 records
Linked 30 records
Linked 40 records
Linked 50 records
Linked 60 records
Linked 70 records
Linked 80 records
Linked 90 records
Linked 100 records
Linked 110 records
Linked 120 records
Linked 130 records
Linked 140 records
Linked 150 records
Linked 160 records
Linked 170 records
Linked 180 records
Linked 190 records
Linked 200 records
Linked 210 records
Linked 220 records
Linked 230 records
Linked 240 records
Linked 250 records
Linked 260 records
Linked 270 records
Linked 280 records
Linked 290 records
Linked 300 records
Linked 310 records
Linked 320 records
Linked 330 records
Linked 340 records
Linked 350 records
Linked 360 records
Linked 370 records
Linked 380 records
Linked 390 records
Linked 400 records
Linked 410 records
Linked 420 records
Linked 430 records
Linked 440 records
Linked 450 records
Linked 460 records
Linked 470 records
Linked 480 records
Linked 490 records
Linked 500 records
Linked 510 records
Linked 520 records
Link

In [ ]:
statsByLocation = pd.DataFrame(columns).dropna()
statsByLocation.reset_index(drop=True, inplace=True)

In [ ]:
identifier = CAN_path.split("/")[1].split("_")[0] + "_" + CAN_path.split("/")[1].split("_")[1]
statsByLocation.to_csv("../decodedDataByLocation_" + identifier + ".csv", index=False)